# New Road Analysis  
We present two approaches:  
1. Geometrical, by comparing the historical and the new activity path segments.
2. Lexical, by comparing the historical and the new activity Strava segments names.

We start by building the athlete historical activities. For the first approach, each activities path is sampled uniformly (by 500m by default).  

In [1]:
%load_ext autoreload
%autoreload 2

First, we set up the Strava client. It's a bit annoying to get the refresh token that has a short life span by default. 

In [2]:
from wandrer.client import StravaClient
strava_credentials = StravaClient()
strava_credentials.configure_client()
activities = strava_credentials.get_activities()

We can build the historical activities datasets (with `fetch_path_segments`) before indexing it in a `BallTree`.  
By default, we only fetch the last 10 activities (`limit`).

In [10]:
from wandrer.activity import HistoricalActivities

history = HistoricalActivities(strava_client=strava_credentials, limit=10)
history.fetch_path_segments()
history.index()

No lat long data for this activity


We can visualize all historical path segments.

In [4]:
import folium

folium_map = folium.Map([48.862, 2.346], zoom_start=11)
markers = [folium.Marker(loc, icon=None, popup="").add_to(folium_map) for loc in [p for segment in history.path_segments for p in segment]]
folium_map

For a new activity, we can compare its path segments with the historical ones as follows. We then display its new segments if any.

In [11]:
from wandrer.activity import Activity

new_activity = Activity(activity_id = activities[-1].id, strava_client=strava_credentials)
new_segments = new_activity.get_new_path_segments(history=history)

folium_map = folium.Map(new_segments[0][0], zoom_start=11)
markers = [folium.Marker(loc, icon=None, popup="").add_to(folium_map) for loc in [p for segment in new_activity.path_segments for p in segment]]

folium_map

Indeed, it's the only time I went to Provins.  

## Simply comparing Strava segments directly

In [39]:
from wandrer.activity import HistoricalActivities
from wandrer.activity import Activity

history = HistoricalActivities(strava_client=strava_credentials, limit=10)
history.fetch_strava_segments()

new_activity = Activity(activity_id = activities[-1].id, strava_client=strava_credentials)
new_activity.fetch_strava_segments()

new_strava_seg = new_activity.get_new_strava_segments(history=history)
print(new_strava_seg)

In [47]:
folium_map = folium.Map([48.862, 2.346], zoom_start=11)
for segment in new_strava_seg:
    folium.Marker(segment['start_latlng'], popup=segment['name']+" Start").add_to(folium_map)
    folium.Marker(segment['end_latlng'], popup=segment['name']+" End").add_to(folium_map)

folium_map

Perhaps that's not an entirely fair comparison, as the geometrical approach benefits from the filtering of the nearest neighbours while the other brute forcely checks the strava segments information as strings directly.